In [1]:


import sys

sys.path.append("D:/StateOfStates/")

import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score

from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian as Partisian
from sklearn.linear_model import LinearRegression
import addfips
import CauseOfDeathData


In [159]:
def cod_timeseries(state = "Michigan", cod = "Suicide"):
    df = CauseOfDeathData.load_state_cod(state="All", year="All", cod = cod)
    df = df.sort_values(by=["year", "start_age"])
    state_df = df[df["state"] == state]
    
    national_avg = df.groupby(["year"]).agg({"deaths": "sum", "population": "sum"}).reset_index()
    national_avg["rate"] = national_avg["deaths"] / national_avg["population"] * 100000
    
    state_avg = state_df.groupby(["year"]).agg({"deaths": "sum", "population": "sum"}).reset_index()
    state_avg["rate"] = state_avg["deaths"] / state_avg["population"] * 100000
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x = national_avg["year"],
        y = national_avg["rate"],
        mode = "lines",
        name = "National Avg",
        line=dict(
            width=4,
        )))

    fig.add_trace(go.Scatter(
        x = state_avg["year"],
        y = state_avg["rate"],
        mode = "lines",
        name = f"{state} Avg",
        line=dict(
            width=4,
        )))
    fig.update_layout(
        autosize=False,
        width=850,
        height=200,
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=0
        ),)
    fig['layout']['xaxis']['title']=f"{cod} Mortality Rate Over Time"
    fig.show()

In [165]:
cod_timeseries(state="New York", cod="Homicide") 

In [ ]:
states = { 'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'
}


states = {v: k for k, v in states.items()}
states


In [ ]:
def translate_ICD10(x, conversion):
    try:
        return conversion[x]
    except:
        return x
    
    



def clean_state_data(df):
    af = addfips.AddFIPS()
    try:
        df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])
    except: 
        df = df.dropna(subset=["State", "Population"])
    #df = df[df["Crude Rate"] != "Unreliable"]
    df.Population = df.Population.astype(float)
    df["Rate"] = (df["Deaths"]/df["Population"])*100000
    df["Rate"] = df["Rate"].round(2)
    df["fips"] = df.apply(lambda x: af.get_state_fips(x["State"]), axis=1)
    df["Year"] = 2019
    df["State_Abv"] = df.apply(lambda x: states[x["State"]], axis=1)
    df = df.sort_values(by=["Rate"], ascending=False)
    try:
        df.drop(columns=["Notes", "State Code", ], inplace=True)
    except:
        None
   
    try:
        f = open("data/Life/Deaths/ICD10Translation.json")
        translation = json.load(f)
        df["ICD-10 Common"] = df["ICD-10 113 Cause List"].apply(lambda x: translate_ICD10(x, translation))
        df.rename(columns={"ICD-10 Common":"Cause Of Death"}, inplace=True)
    except:
        None
    return df
    


def top_n(df, n):
    #Filter out low occuring chapters
    df = df.groupby("ICD-10 113 Cause List Code").filter(lambda x: len(x)>1)
    df = df.sort_values(by = ["Rate"], ascending = False)
    df = df.head(n).copy()
    top_codes = list(df["ICD-10 113 Cause List Code"].unique())
    df = df[df["ICD-10 113 Cause List Code"].isin(top_codes)]
    return df


firearms = pd.read_csv(r"../data/Life/Deaths/FireArmState.txt", delimiter="	", na_values = ['Not Applicable'])
df = pd.read_csv(r"../data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = pd.read_csv(r"../data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = clean_state_data(state_pop)
yearly_pop = pd.read_csv(r"../data/Life/Deaths/2019State1YearPop.txt", delimiter="	", na_values = ['Not Applicable'])
state_deaths = pd.read_csv(r"../data/Life/Deaths/2019StateDeaths.txt", delimiter="	", na_values = ['Not Applicable'])


df = clean_state_data(df)
firearms = clean_state_data(firearms)
state_pop = clean_state_data(state_pop)
yearly_pop = clean_state_data(yearly_pop)
state_deaths = clean_state_data(state_deaths)
state_pop["Total Pop"] = state_pop["Population"]

#top_df = top_n(df, 5)
#county = df.groupby(by=["2006 Urbanization Code", "ICD Sub-Chapter"]).mean().reset_index().sort_values(by=["Rate"], ascending=False)
state_pop.sample(5)
firearms = firearms.merge(state_pop[["State", "Total Pop"]], left_on="State",right_on="State")

In [ ]:
state_pop

In [ ]:
part = Partisian.get_state_part_score()
final = pd.merge(part, firearms, on="fips")
final[final["State_x"] == "VT"]
state_totals = final.groupby(by=["fips","State_y"]).mean().reset_index().sort_values(by="Rate")
state_totals["Rate"] = (state_totals["Deaths"]/state_totals["Total Pop"])*100000

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Part_Score"], y=state_totals["Rate"], mode = "markers",  hovertext=state_totals["State_y"], marker=dict(size=state_totals["Total Pop"]/1000000, colorscale='bluered', color=state_totals["Part_Score"])))

In [ ]:
state_totals.sort_values("Rate", ascending=False, inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(x=state_totals["State_y"], y=state_totals["Rate"], name="Rate"))
fig.show()

In [ ]:
state_totals[state_totals["State_y"] == "Vermont"]

In [ ]:
state_totals

In [ ]:
12/133915.0

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Population"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [ ]:
usa_total = yearly_pop.groupby("Single-Year Ages").sum().reset_index()
usa_total["age"] = usa_total["Single-Year Ages"].str.replace("< 1 year","0").str.split(" ").str[0].astype(int)
usa_total.sort_values("age", inplace=True)
usa_total["Rate"] = (usa_total["Deaths"]/usa_total["Population"])*100000
usa_total

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Population"]))
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Rate"]))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=usa_total["age"], y=usa_total["Rate"], mode="lines"))

In [ ]:
df["Ten-Year Age Groups Code"].unique()

In [ ]:
child = state_deaths[state_deaths["Ten-Year Age Groups Code"] == '5-14']
part = Partisian.get_state_part_score()
child = pd.merge(child, part, left_on="State_Abv", right_on="State")


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=child['Part_Score'], y=child['Rate'],mode='markers',name='Child'))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=child["State_Abv"], y=child["Rate"], marker=dict(color=child["Part_Score"], colorscale="Bluered" ), name="Child"))

In [ ]:
child

In [ ]:
import GraphGenerator
def get_age_death(df, age):
    age_df = df[df["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    X = age_df['Part_Score'].values.reshape(-1,1)
    y = age_df['Rate']
    
    reg = LinearRegression().fit(X, y, sample_weight=age_df['Population'])
    x_range = np.linspace(X.min(), X.max())
    y = reg.predict(x_range.reshape(-1,1))
    
    fig = make_subplots(rows=1, cols=2, vertical_spacing= 0.01, horizontal_spacing= 0.01,  subplot_titles=("",""), column_widths=[0.8, 0.2], shared_yaxes=True)
    
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers', marker=dict(color=age_df["Part_Score"], colorscale="Bluered", size=age_df["Population"]/(age_df["Population"].max()/50)), name='Child', hovertext=age_df["State_x"]), row=1, col=1)
    
    hist_x = []
    for index, row in age_df.iterrows():
        hist_x.extend([row["Rate"] for i in range(int(round(row["Population"]/1000)))])
        row["Rate"]
    #fig.add_trace(go.Violin(y=hist_x, side="positive", name="Death Rate Distribution"), row=1, col=2)
    fig.add_trace(go.Violin(y=age_df['Rate'], side="positive",  name="Death Rate Distribution"), row=1, col=2)
    
    
    fig.add_trace(go.Scatter(x=x_range, y=y, mode='lines',  line=dict(color="green", dash="dash"), name='Best Fit Line'), row=1, col=1)
    
    fig.update_layout(title="Partisanship Score vs. Death Rate for " + age + " Year Olds", xaxis1_title="Partisanship Score", yaxis1_title="Death Rate (per 100,000)")
    fig.update_yaxes(showticklabels=False, row=1, col=2)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
    fig.update_layout(height=300,width=1200, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=25, b=0))
    
    fig.show()


In [ ]:
get_age_death(state_deaths,"85+")

In [ ]:
for age in state_deaths["Ten-Year Age Groups Code"].unique():
    None
    #get_age_death(state_deaths,age)

In [ ]:

fig = go.Figure()

for age in state_deaths["Ten-Year Age Groups Code"].unique():
    age_df = state_deaths[state_deaths["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers',name=age))
    #fig.add_trace(go.Bar(x=age_df["State_Abv"], y=age_df["Rate"], marker=dict(color=age_df["Part_Score"], colorscale="Bluered" ), name=age))

fig.show()

In [ ]:
teen = df[df["Ten-Year Age Groups Code"] == "85+"]
teen.sample(3)
teen_all = teen#[teen["ICD-10 Original"].str.contains("#") == True]
ny = teen_all[teen_all["State"] == "New York"]
ny.dropna(inplace=True, subset=["ICD-10 113 Cause List"])
ny.tail(3)

In [ ]:
f = open("icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [ ]:
#ny.loc[15081]

In [ ]:
ny["ICD-10 113 Cause List"] = pd.Categorical(ny["ICD-10 113 Cause List"], categories = list(heirarchy.keys()))
ny = ny.sort_values(by=["ICD-10 113 Cause List"])
ny["ICD-10 113 Cause List"].unique()
ny.dropna(subset=["ICD-10 113 Cause List"], inplace=True)

In [ ]:
ny

In [ ]:
labels = ["All"]
labels.extend(list(ny["ICD-10 113 Cause List"]))

values = [0]
values.extend(list(ny.Rate))


parents = []
for i,label in enumerate(labels):
    if label == "All":
        parents.append("")
    else:
        try:
            parents = flat_heirarchy[label]["parents"]
            if len(parent) > 0:
                parents.append(parent[0])
            else:
                parents.append("")
                #values[i] = values[i]-values[parents.index(heirarchy[label])]
            
        except:
            parents.append("")  



In [ ]:
test_labels = labels[:11]
test_parents = parents[:11]
test_values = values[:11]
test_values[0] = sum(test_values[1:])

In [ ]:
fig =go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
))

fig.show()

In [ ]:
df = pd.read_csv(r"../data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])

In [ ]:
df["Ten-Year Age Groups Code"].unique()

In [ ]:
f = open("../icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("../icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [ ]:
def load_state_cod(state="Michgian"):
    f = open("../icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    df_heirarchy = pd.DataFrame(flat_heirarchy)
    #rotate Matrix
    df_heirarchy = df_heirarchy.transpose()
    df_heirarchy.reset_index(inplace=True)
    df_heirarchy.rename(columns={"index":"ICD-10 113 Cause List"}, inplace=True)
    df_heirarchy

    df = pd.read_csv(r"../data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    
    #state_df = state_df[state_df["Ten-Year Age Groups Code"] ==age]
    df["Rate"] = (df["Deaths"] / df["Population"])*100000
    df["Rate"] = np.round(df["Rate"], 2)
    df["Child_Adj_Rate"] = df["Rate"]
    if(state != "All"):
        df= df[df["State"] == state]


    df_final = pd.merge(df, df_heirarchy, on="ICD-10 113 Cause List")
    df_final = df_final.drop(columns=["ICD-10 113 Cause List Code", "ranges", "chapters", "base", "Notes"])
    return df_final

In [ ]:
def cod_v_age_box(cod, state):
    df = load_state_cod("All")
    df = df[df["ICD-10 113 Cause List"] == cod]
    df = df.sort_values(by=["Ten-Year Age Groups Code"])
    # Create box plot  
    fig = px.box(df, x="Ten-Year Age Groups Code", y="Rate", color="Ten-Year Age Groups Code", hover_data=["State", "ICD-10 113 Cause List"])
    fig.show()
    

In [ ]:
cod_v_age_box("Renal failure (N17-N19)", "Michigan")

In [ ]:
df


In [ ]:
def get_age_box(cod, state):
    df = load_state_cod("All")
    df = df[df["ICD-10 113 Cause List"] == cod]
    df["Start Age"] = df["Ten-Year Age Groups Code"].str.split("-").str[0].str.replace("+","")
    df["Start Age"] = df["Start Age"].astype(int)

    df = df.sort_values(by=["Start Age"])

    fig = px.box(df, x="Ten-Year Age Groups Code", y="Rate", color="Ten-Year Age Groups Code", hover_data=["State", "ICD-10 113 Cause List"])

    df = df[df["State"] == state]
    fig.add_trace(go.Scatter(
        x = df["Ten-Year Age Groups Code"],
        y=df["Rate"],
        mode = "markers+lines",
        marker=dict(color = "red"),
        name = state

    ))
    fig.update_layout(height=300,width=1000, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=25, b=0))

    fig.show()

In [ ]:
get_age_box("#Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)", "Michigan")

In [ ]:
df.sort_values(by="Start Age")

In [ ]:
def add_children(name, child, order):
    f = open("../icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    order.append(name)
    if len(child["children"]) > 0: 
        for grand_child in child["children"]:
            add_children(grand_child, flat_heirarchy[grand_child], order)
def get_depth_order(flat_heirarchy):
    order = []
    for key, value in flat_heirarchy.items():
        if len(value["parents"]) == 0:
            order.append(key)
            for children in value["children"]:
                add_children(children, flat_heirarchy[children],order)
    return order

In [ ]:
def get_age_cod(df, age, all_ages=False):
    f = open("../icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    order = get_depth_order(flat_heirarchy)
    df = df[df["Ten-Year Age Groups Code"] ==age]
    for e in range(5):
        for i, row in df.iterrows():
            children_total = 0
            children_values =[]
            if len(row["children"]) > 0:
                for child in row["children"]:
                    try:
                        children_total += df[df["ICD-10 113 Cause List"] == child]["Child_Adj_Rate"].values[0]
                        children_values.append(df[df["ICD-10 113 Cause List"] == child]["Child_Adj_Rate"].values[0])
                    except:
                        None
            df.at[i, "Child_Adj_Rate"] = max(row["Child_Adj_Rate"], children_total)
    df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].astype("category")
    df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].cat.set_categories(order, ordered=True)
    df = df.sort_values("ICD-10 113 Cause List")
    
    labels = [age]
    parents = [""]
    if all_ages:
        parents = ["All Ages"]
    values = [0]
    colors = ["white"]
    for i, row in df.iterrows():
        labels.append(row["ICD-10 113 Cause List"].split("(")[0] + ": " + str(age))
        if len(row["parents"]) > 0:
            parents.append(row["parents"][0].split("(")[0] + ": " + str(age))
        else: 
            parents.append(age)
            values[0] += row["Child_Adj_Rate"]
        values.append(row["Child_Adj_Rate"])
    return labels, parents, values, df, values[0]
    
df_final = load_state_cod()
labels, parents, values, df, total = get_age_cod(df_final, age, all_ages=True)

In [ ]:
def get_all_age_cod(df):
    all_labels = ["All Ages"]
    all_parents = [""]
    all_values = [0]
    for age in df["Ten-Year Age Groups Code"].unique():
        labels, parents, values, df_age, total = get_age_cod(df, age, True)
        all_labels.extend(labels)
        all_parents.extend(parents)
        all_values.extend(values)
        all_values[0] += total
    return all_labels, all_parents, all_values
    
final_labels, final_parents, final_values = get_all_age_cod(df_final)    

In [ ]:
fig = go.Figure(go.Treemap(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
    textinfo = "label+value+percent root",
    textposition = 'middle center',
    textfont = dict(family="Arial", size=16, color = '#FFFFFF')
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
    
)

fig.show()
fig.write_html("CauseOfDeathTest.html")

In [ ]:
fig = go.Figure(go.Sunburst(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()

In [ ]:
fig = go.Figure(go.Icicle(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
    tiling = dict(
            orientation='v',
        ),
    textposition = 'middle center',
    textfont = dict(family="Arial", size=18, color = '#FFFFFF')
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    #uniformtext=dict(minsize=8, mode='hide'),
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()
fig.write_html("CauseOfDeathIcicle.html")

#here

In [ ]:
def load_state_cod_old(state, age_group = '75-84'):
    df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    state_df= df[df["State"] == state]
    state_df = state_df[state_df["Ten-Year Age Groups Code"] =='75-84']
    state_df["Rate"] = (state_df["Deaths"] / state_df["Population"])*100000
    state_df["Rate"] = np.round(state_df["Rate"], 2)
    state_df["Child_Adj_Rate"] = state_df["Rate"]
    
    state_df["Parent"] = ""
    state_df["Children"] = ""
    state_df["Common Name"] = ""
    
    
    f = open("icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)

    targets =[]
    sources = []
    values = []

    sources.append("")
    targets.append("All")
    values.append(0)
    values.extend(list(state_df.Rate))
    


    for i, row in state_df.iterrows():
        try:
            structure = flat_heirarchy[row["ICD-10 113 Cause List"]]
            state_df.at[i, "Common Name"] = structure["common_name"]
            targets.append(structure["common_name"])
            structure["Rate"] = row["Rate"]
            
            if len(structure["parents"])  == 1:
                parent = flat_heirarchy[structure["parents"][0]]["common_name"]
                state_df.loc[i, "Parent"] = parent
                sources.append(parent)
            else:
                state_df.loc[i, "Parent"] = "All"
                sources.append("All")
        except Exception as e:
            print(e)
        if len(structure["children"]) > 0:
            state_df.at[i, "Children"] = structure["children"]
             
    for i, source in enumerate(sources):
        if source == "":
            None
        else:
            try:
                parent_index = targets.index(source)
                values[parent_index] = max( values[parent_index] - values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[parent_index]
                
            except:
                values[i] = max(values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[i]
                
    values[0] = 5000
    size = 50
    plot_dict = {"values":values[:size], "targets":targets[:size], "parents":sources[:size]}
    return state_df, plot_dict

In [ ]:
df_final = load_state_cod(state="All")
state_pop = pd.read_csv(r"../data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = clean_state_data(state_pop)[["State","Population","Deaths","State_Abv"]]
state_pop.rename(columns={"Deaths":"Total Deaths"}, inplace=True)

In [ ]:
cardio = df_final[df_final["ICD-10 113 Cause List"] == "Major cardiovascular diseases (I00-I78)"].groupby("State").agg({"Deaths":"sum"}).reset_index()
cardio = pd.merge(cardio, state_pop, left_on="State", right_on="State")
cardio["Rate"] = cardio["Deaths"]/cardio["Population"]*100000
cardio = cardio.sort_values("Rate", ascending=False)
cardio

In [ ]:
def nation_single_cod(cod = "Major cardiovascular diseases (I00-I78)", state = "Michigan"):
    # Load Data
    df = pd.read_csv(r"..\data\Life\Deaths\2019CuaseOfDeath.txt", delimiter="	", na_values = ['Not Applicable'])
    df = clean_state_data(df)
    # Filter by COD
    cod = cod.split(":")[0].strip()
    df = df[df["ICD-10 113 Cause List"].str.contains(cod)].sort_values("Rate", ascending=False)
    
    
    fig = go.Figure()
    fig.add_trace(go.Box(
        x=df["Rate"], 
        name=cod.split("(")[0],
        boxpoints='all', # can also be outliers, or suspectedoutliers, or False
        jitter=0.3, # add some jitter for a better separation between points
        pointpos=-1.8, # relative position of points wrt box
        hovertext=df["State_Abv"]

    ))
    fig.add_vline(x=df[df["State"]==state]["Rate"].values[0], line_width=3, line_dash="dash", line_color="green")
    fig.show()
    return df


In [ ]:
cancer = nation_single_cod(cod = "#Malignant neoplasms : 85+", state="Texas")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(
    x=cancer["Rate"], 
    name="Cancer",
    boxpoints='all', # can also be outliers, or suspectedoutliers, or False
    jitter=0.3, # add some jitter for a better separation between points
    pointpos=-1.8, # relative position of points wrt box
    hovertext=cancer["State_Abv"]
            
))
fig.add_vline(x=226.99, line_width=3, line_dash="dash", line_color="green")


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Violin(
    x=cancer["Rate"], 
    name="Cancer",
    points='all', # can also be outliers, or suspectedoutliers, or False
    jitter=0.3, # add some jitter for a better separation between points
    pointpos=-1.8, # relative position of points wrt box
    hovertext=cancer["State_Abv"],
    side="positive"
            
))
fig.add_vline(x=226.99, line_width=3, line_dash="dash", line_color="green")


fig.show()

In [ ]:
cancer